In [10]:
import tensorflow as tf
import os
from pathlib import Path
import urllib.request as request
import numpy as np
import matplotlib.pyplot as plt

In [3]:
IMAGE_SIZE = (224, 224)

In [5]:
model = tf.keras.applications.VGG16(
    include_top=True, weights="imagenet", input_tensor=None, input_shape=None, classes=1000 
)  

In [6]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [7]:
model.save("VGG16_full_model.h5")

In [8]:
model.layers[1].padding

'same'

In [5]:
data_URL = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"

# create a directory
def create_dirs(dir_path):
    os.makedirs(dir_path, exist_ok=True)
    print(f"{dir_path} directory created")
    
ROOT_DATA_DIR = "data"
create_dirs(ROOT_DATA_DIR)


data_zip_file = "data.zip"
data_zip_path = os.path.join(ROOT_DATA_DIR, data_zip_file)

if not os.path.isfile(data_zip_file):
    print("downloading data...")
    filename, headers = request.urlretrieve(data_URL, data_zip_path)
    print(f"filename: {filename} created with info \n{headers}")
else:
    print(f"file is already present")

data directory created
downloading data...
filename: data\data.zip created with info 
Content-Type: application/zip
Content-Length: 47286322
Connection: close
Last-Modified: Wed, 15 Mar 2017 18:46:00 GMT
x-amz-version-id: null
Accept-Ranges: bytes
Server: AmazonS3
Date: Sat, 02 Apr 2022 08:09:46 GMT
ETag: "5f8c32a6554f6acb4d649776e7735e48"
X-Cache: Hit from cloudfront
Via: 1.1 b93822242d240fe957b16155421ce866.cloudfront.net (CloudFront)
X-Amz-Cf-Pop: NRT57-P2
X-Amz-Cf-Id: oDfh5X-WA9UxgaP2Y0WYyWpHo7QyGPJANZI2rCwhJ6wXY1GARabaIg==
Age: 68796


